# Introduction
1. Question Answer pair generation using Llama 2 with few short prompting for RAG evalution
2. Chunking the conversation Data
3. Vector DB intialized
4. RAG Pipline defined
5. Generating answer for the Questions
6. Evalution

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset/conversation.csv')

In [3]:
data.head()

,conversation,conv_id
0,Are you a fan of Google or Microsoft?\nBoth ar...,1
1,do you like dance?\nYes I do. Did you know Br...,2
2,Hey what's up do use Google very often?I reall...,3
3,Hi! do you like to dance?\nI love to dance a ...,4
4,do you like dance?\nI love it. Did you know Br...,5


In [4]:
import transformers
import torch
from transformers import AutoTokenizer

### Pipline Initalization

In [5]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    device_map="auto",
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    max_length=4000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


### Question Answer pair generation using Llama 2 with few short prompting for RAG evalution

As we dont have any question we are generating the question and answer pairs from the first 50 converstation we have generated about 100 qa pairs. To do so we have used Llama 2 fine with few short prompting.

In [6]:
from collections import defaultdict
from tqdm import tqdm
import ipdb

q = defaultdict(list)
a = defaultdict(list)
for i in tqdm(range(1,50)):
    prompt = f"""<s>[INST] Generates only 3 question and answer pairs bases on the converstation provided in triple back ticks ```{data.conversation[0]}``` \
use <question> and <ans> tags to indicate them:

<question> Do you like cats?
<ans> The cat is referred as domestic cat and wild cat. They make our world very clean from rats!
<question> Why do you think cats spend a lot of their time sleeping?
<ans> Cats hear the sounds too faint or too high frequency human ears can hear.
<question> What are some of the services provided by Google?
<ans> Google provides online related services and products, which includes online ads, search engine and cloud computing.
[/INST]</s>

<s>[INST] Generates only 3 question and answer pairs bases on the converstation provided in triple back ticks ```{data.conversation[i]}``` \
use <question> and <ans> tags to indicate them:

"""
    seq = pipeline(prompt) 
    x = seq[0]['generated_text'].split(prompt)
    for elm in x[1].split("\n"):
        if elm.find("<question>")!=-1:
            q[i+1].append(elm)
        elif elm.find("<ans>")!=-1:
            a[i+1].append(elm)
    # print(q,a)
    # ipdb.set_trace()

 20%|████████████████████████▋                                                                                                | 10/49 [01:39<06:23,  9.84s/it]/home2/sisodiya.bhoomendra/venvs/python3.9_global/lib/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [07:28<00:00,  9.16s/it]


In [7]:
ques,anss,conv_id = [],[],[]
for k in q.keys():
    for que,ans in zip(q[k],a[k]):
        if ans is not None:
            ques.append(que)
            anss.append(ans)
            conv_id.append(k)

pd.DataFrame({"Question":ques,"Answer":anss,"Conv_id":conv_id}).to_csv("dataset/gen_qa_pair.csv",index=False)

### Chunking the converstions
A coversation is made of messages and we have made chucks of 6 messages i.e a combination of 6 messages will be used to get an embedding in the vector DB.

In [52]:
docs = []
cov_ids = []

for _,x in data.iterrows():
    
    lines = x.conversation.split("\n")
    for i in range(0,len(lines),6):
        docs.append("\n".join(lines[i:i+6]).strip("\n"))
        cov_ids.append(str(x.conv_id))


In [53]:
len(docs),len(cov_ids)

(409, 409)

### Vector DB intialized

We initalized the vector DB with the chucked conversations also we have added the coversation id as this will help us in refering to the document which led to the answer

In [54]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="my_collection")
collection.add(
    documents=docs,
    ids=cov_ids
)

Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


### RAG Pipline defined

In [70]:
def rag_pipeline(question):
    results = collection.query(query_texts=[question],n_results=2)
    
    prompt = f"<s> [INST] Aswer the following question in few words \n \
    Question: {question} \n Basesd on the Context:\
    {results['documents'][0][0]} \n {results['documents'][0][1]}  [/INST]\n Answer: "
    
    seq = pipeline(prompt) 
    x = seq[0]['generated_text'].split(prompt)
    
    return (x[1],",".join(results['ids'][0]).strip(','))

In [68]:
qa_pair = pd.read_csv("dataset/gen_qa_pair.csv")

### Generating answer for the Questions

In [71]:
gen_ans = []
relevent_cov = []

for _,r in tqdm(qa_pair.iterrows()):
    q = r.Question.strip("<question> ")
    ans,rel_conv = rag_pipeline(q)
    gen_ans.append(ans)
    relevent_cov.append(rel_conv)

qa_pair['gen_ans'] = gen_ans
qa_pair['retrived_cov_ids'] = relevent_cov

qa_pair.to_csv("dataset/gen_qa_ans.csv",index=False)

97it [04:45,  2.94s/it]


## Evalution

In [8]:
from evaluate import load

rouge = load('rouge')
bertscore = load('bertscore')

In [9]:
df = pd.read_csv("dataset/gen_qa_ans.csv")

In [10]:
df.head()

,Unnamed: 0,Question,Answer,Conv_id,gen_ans,retrived_cov_ids
0,0,<question> Do you like dance?,"<ans> Yes, I do.",2,"Yes, I like dance. Bruce Lee was a dancer too...","5,61"
1,1,<question> Did you know Bruce Lee was a cha ch...,"<ans> Yes, he even won a hardcore cha cha cham...",2,Bruce Lee was a cha cha dancer and won a cham...,"2,64"
2,2,<question> Did you know Tupac was a ballet dan...,"<ans> Yes, and he even was in the production o...",2,"Tupac was a ballet dancer? No, I didn't know ...","64,2"
3,3,<question> Do ballet dancers go through a lot ...,"<ans> Yes, they go through 4 pairs of shoes a ...",2,"Yes, ballet dancers go through many shoes, of...","2,4"
4,4,<question> Did you know babies are good at dan...,"<ans> Yes, and they smile more when they hit t...",2,"Yes, babies are good at dancing. They spontan...","65,2"


In [11]:
from tqdm import tqdm

In [15]:
def evaluate_QA():
    rouge_1,rouge_score,bert_score = 0,0,0
    for _,x in tqdm(df.iterrows()):
        ref = [x.Answer.strip("<ans> ")]
        pred = [x.gen_ans]
        r = rouge.compute(references=ref,predictions=pred)
        rouge_score +=r['rougeL']
        rouge_1+=r['rouge1']
        out = bertscore.compute(references=ref,predictions=pred, lang="en")
        bert_score += out['f1'][0]
    print(f"The evalutation of are as follows :\n*Rouge1: {rouge_1/len(df)} \n*RougeL: {rouge_score/len(df)}\n*BertScore: {bert_score/len(df)}")
evaluate_QA()

97it [00:21,  4.61it/s]

The evalutation of are as follows :
*Rouge1: 0.2681507432372196 
*RougeL: 0.2287766904620272
*BertScore: 0.8748656437569058


### Results and conclusion

The task was quit open ended and hence evalution becomes hard as the question but we have done the task in the following manner. First we generated the QA pair from the Llama 2 model this will serve a benchmark to evalute our RAG pipline. Then we build RAG pipline to perform QA over the conversation which was evaluted with the generated question ans pairs.

|Rouge1|RougeL|BertScore|
|-|-|-|
|0.2681|0.2287|0.8748|

# Future Works
1. Try different chunikng methods
2. Generate a meaning QA dataset from the conversation 
3. Ones we have some dataset we can try few short prompting and fine tuning 